In [1]:
from base_module import *

action_encoding

/data/satori_hdd4/Ren/envs/mahjong_satori/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'player0': {'discard': {'0m': 0,
   '1m': 1,
   '2m': 2,
   '3m': 3,
   '4m': 4,
   '5m': 5,
   '6m': 6,
   '7m': 7,
   '8m': 8,
   '9m': 9,
   '0p': 10,
   '1p': 11,
   '2p': 12,
   '3p': 13,
   '4p': 14,
   '5p': 15,
   '6p': 16,
   '7p': 17,
   '8p': 18,
   '9p': 19,
   '0s': 20,
   '1s': 21,
   '2s': 22,
   '3s': 23,
   '4s': 24,
   '5s': 25,
   '6s': 26,
   '7s': 27,
   '8s': 28,
   '9s': 29,
   '1z': 30,
   '2z': 31,
   '3z': 32,
   '4z': 33,
   '5z': 34,
   '6z': 35,
   '7z': 36},
  'chi_left': 37,
  'chi_middle': 38,
  'chi_right': 39,
  'pon': 40,
  'minkan': 41,
  'ankan': 42,
  'kakan': 43,
  'reach': 44,
  'agari': 45,
  'pass_naki': 46,
  'pass_reach': 47,
  'kyushukyuhai': 48},
 'player1': {'discard': {'0m': 49,
   '1m': 50,
   '2m': 51,
   '3m': 52,
   '4m': 53,
   '5m': 54,
   '6m': 55,
   '7m': 56,
   '8m': 57,
   '9m': 58,
   '0p': 59,
   '1p': 60,
   '2p': 61,
   '3p': 62,
   '4p': 63,
   '5p': 64,
   '6p': 65,
   '7p': 66,
   '8p': 67,
   '9p': 68,
   '0s': 69,
   